#### Week 1: Building Advanced RAG Applications.  Authored by Chris Sanchez.

# Week 1 - Notebook 3

# Overview
***
Welcome to the final notebook for Week 1! Take a look at all the ground we've covered so far:
- Chunking/splitting
- Text vectorization
- Combining metadata
- Collection Configuration
- Data Indexing
- Keyword search
- Vector search
- OPTIONAL: Searching with Filters

We are now prepared to move on to a very important topic, **Retrieval Evaluation**.  I hope you've noticed that the search results will differ (sometimes slightly, sometimes by a lot) depeding on which search method you used: `keyword_search` or `vector_search`.  As humans, it's fairly easy for us to determine whether the returned search results are relevant to the query that was submitted, (though even here there will be differing opinions on result relevance).  But how do we systematically determine which search method is better in general?  And how do we measure the relative performance of our retrieval system if we change one of it's parameters...for example, changing our embedding model? What about measuring system performance over time as more documents are added to our datastore?

We need a way to evaluate our retrieval system, and this notebook will show you "one way" of doing that.  I say "one way" because there are many ways to approach this problem, and the method I'm showing you is not perfect (if anything it's a bit too conservative).  Ultimately, measuring retrieval performance is hard because it requires a lot of time and effort, and absent any user [click-data](https://en.wikipedia.org/wiki/Click_tracking), requires some form of data labeling.  With the advent of powerful generative LLMs the process of measuring retrieval performance has become much easier. Let's take a look at how that works.

# Retrieval Evaluation - Process
***
Here's a high-level overview of how the Retrieval Evaluation process in this notebook works:

1. Generate a "golden dataset" of query-context pairs.  I've already pre-generated multiple golden datasets for our course. I randomly selected 100 document chunks (contexts) from the Huberman Lab corpus and those chunks were then submitted to the `gpt-3.5-turbo` model which generated queries that can be answered by the context.  The output was 100 query-context pairs along with their associated doc_ids. 
   - **Baseline Assumptions**:
     - The generated query-context pairs are, in fact, relevant to one another i.e. the query can be answered by the context that it's paired with
     - The generated queries are simliar in style and length to the type of queries that end users would ask
2. The golden dataset consists of three primary keys: `corpus`, `relevant_docs`, and `queries`
     - The `corpus` is the original text context/chunk with it's associated `doc_id`
     - The `queries` are the LLM generated queries, one (or more) for each entry in the `corpus`
     - The `relevant_docs` is a simple lookup table linking the `corpus` docs to the generated `queries`
3. We pass the golden dataset into a retrieval evluation function which does the following:
   - Takes in a `retriever` arg (`WeaviateWCS`) and a few other configuration params
   - Iterates over all queries in the golden dataset and retrieves search results for each query from the Weaviate datastore
   - Extracts all `doc_id` values from the retrieved results
   - Extracts the `doc_id` from the associated `relevant_docs` for each query
   - Checks if the relevant doc_id is in the list of retrieved result doc_ids
   - After all queries are completed a `hit_rate` score and `mrr` score are calculated for the entire golden dataset
   - Writes results to an `eval_results` folder

#### In a Nutshell
Ulitmately, given a golden dataset consisting of queries, relevant docs, and their associated doc_ids, the `retrieval_evaluation` function is checking if the relevant `doc_id` is found in the list of retrieved results `doc_ids`, for each query.

#### Problems with this Approach
The problems with this approach are many, I'll cover a few here:
- The **Assumptions** (see section 1 above) about the golden dataset must hold true.  Given that the pairs are generated by `gpt-3.5-turbo`, I think the first assumption will generally be true.  When reviewing the dataset I did find a few questions that were not answerable given the context, but for the most part they were.  The 2nd assumption though, is going to be dependent on your particular search use case.  I think for the purposes of this course, the questions generated are a decent reflection of how someone would query this dataset, and therefore do the job of measuring retriever performance.  But I would always check a real-world query distribution before using an approach like the one presented here.
- This approach is conversative in that there is only "one" right answer.  Either the relevant `doc_id` is in the results list or it isn't.  In reality, there are going to be several documents that could potentially answer the generated query, but we would have to use other relevance evalauation methods to account for these documents, unless of course, we want to manually add doc_ids to the golden dataset (and depending on your business case, you may actually want to do that).
- We aren't measuring recall or precision because we aren't classifying other documents as "negatives".  As was just mentioned, the other documents in the results list may or may not be good matches, we just don't know.  Because we don't know, we can't really classify the other documents as "negatives".  So for this approach, we are measuring the ["hit rate"](https://uplimit.com/course/rag-applications/session/session_clzlsa20a01di197e4tij7vgm/module/module_clzlsa5qo02bh1dm698ip60sm#corise_clp66zqui003j2a77u8lnrk5b) which is simply a count of the number of times that we found a relevant `doc_id` match in the results list and [Mean Reciprocal Rank (MRR)](https://uplimit.com/course/rag-applications/session/session_clzlsa20a01di197e4tij7vgm/module/module_clzlsa5qo02bh1dm698ip60sm#corise_clp66zqui003j2a77u8lnrk5b).  We're using MRR instead of other metrics such as Mean Average Precision (MAP) because we are only looking at a [single relevant answer](https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-rank).  Hit rate is a good enough metric for determining if our retriever is retrieving quality results, and MRR will become more important later on when we add a Reranker to the mix.  

In [1]:
#standard library imports
import sys
sys.path.append('../')

from typing import Any
import time
import os

# utilities
from tqdm import tqdm
from rich import print
from dotenv import load_dotenv, find_dotenv
env = load_dotenv(find_dotenv(), override=True)

# Assignment 1.3
***
#### Instructions:
* Import the `/data/golden_datasets/golden_256.json` dataset using the `load_json` method of the FileIO Class
* Instantiate a new WeaviateWCS client (Retriever) and set the `collection_name` of the Collection that you created in Notebook 2
* Evaluate your retriever results using the `retrieval_evaluation` function
* Submit your results in the form of a text file to Uplimit (the function autogenerates a report in the `dir_outpath` directory).

In [3]:
from src.database.weaviate_interface_v4 import WeaviateWCS
from src.preprocessor.preprocessing import FileIO


data_path = '../data/golden_datasets/golden_256.json'

#################
##  START CODE ##
#################
api_key = os.environ['WEAVIATE_API_KEY']
url = os.environ['WEAVIATE_ENDPOINT']
model_path = 'sentence-transformers/all-MiniLM-L6-v2'

client = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path)

#example of using the private _client attribute
client._client.is_connected()

### Load QA dataset
golden_dataset = FileIO.load_json(data_path)

### Instantiate Weaviate client and set Collection name
retriever = client
# retriever.show_all_collections
collection_name = "Huberman_minilm_256"


#################
##  END CODE   ##
#################

# should see 100 queries
print(f'Num queries in Golden Dataset: {len(golden_dataset["queries"])}')

Num queries in Golden Dataset: 100

# Project 1: Retrieval Evaluation

In [4]:
from src.evaluation.retrieval_evaluation import calc_hit_rate_scores, calc_mrr_scores, record_results

def retrieval_evaluation(dataset: dict, 
                         collection_name: str, 
                         retriever: WeaviateWCS,
                         retrieve_limit: int=5,
                         chunk_size: int=256,
                         query_properties: list[str]=['content'],
                         return_properties: list[str]=['doc_id', 'content'],
                         dir_outpath: str='./eval_results',
                         include_miss_info: bool=False
                         ) -> dict[str, str|int|float]:
    '''
    Given a dataset and a retriever evaluate the performance of the retriever. Returns a dict of kw and vector
    hit rates and mrr scores. If inlude_miss_info is True, will also return a list of kw and vector responses 
    and their associated queries that did not return a hit, for deeper analysis. Text file with results output
    is automatically saved in the dir_outpath directory.

    Args:
    -----
    dataset: dict
        Dataset to be used for evaluation
    collection_name: str
        Name of Collection on Weaviate host to be used for retrieval
    retriever: WeaviateWCS
        WeaviateWCS object to be used for retrieval 
    retrieve_limit: int=5
        Number of documents to retrieve from Weaviate host, increasing this value too high 
        will artificially inflate the hit rate score of your retriever.
    chunk_size: int=256
        Number of tokens used to chunk text. This value is purely for results 
        recording purposes and does not affect results. 
    query_properties: list[str] = ['content']
        List of properties over which keyword search will query.  Can add multiple properties
        to this list. 
    return_properties: list[str]=['doc_id', 'content']
        list of properties to be returned from Weaviate host for display in response
    dir_outpath: str='./eval_results'
        Directory path for saving results.  Directory will be created if it does not
        already exist. 
    include_miss_info: bool=False
        Option to include queries and their associated kw and vector response values
        for queries that are "total misses"
    '''

    results_dict = {'n':retrieve_limit, 
                    'Retriever': retriever.model_name_or_path, 
                    'chunk_size': chunk_size,
                    'query_props': query_properties,
                    'kw_raw_hits': 0,
                    'vector_raw_hits': 0,
                    'kw_mrr': 0,
                    'vector_mrr': 0,
                    'total_misses': 0,
                    'total_questions':0
                    }
    
    start = time.perf_counter()
    miss_info = []
    for query_id, q in tqdm(dataset['queries'].items(), 'Queries'):
        results_dict['total_questions'] += 1
        hit = False
        
        try:
            kw_response = retriever.keyword_search(request=q, collection_name=collection_name, query_properties=query_properties,
                                                   limit=retrieve_limit, return_properties=return_properties)
            vector_response = retriever.vector_search(request=q, collection_name=collection_name, 
                                                   limit=retrieve_limit, return_properties=return_properties)
            
            #collect doc_ids and position of doc_ids to check for document matches
            kw_doc_ids = {result['doc_id']:i for i, result in enumerate(kw_response, 1)}
            vector_doc_ids = {result['doc_id']:i for i, result in enumerate(vector_response, 1)}
            
            #extract doc_id for scoring purposes
            doc_id = dataset['relevant_docs'][query_id]
 
            #increment hit_rate counters and mrr scores
            if doc_id in kw_doc_ids:
                results_dict['kw_raw_hits'] += 1
                results_dict['kw_mrr'] += 1/kw_doc_ids[doc_id]
                hit = True
            if doc_id in vector_doc_ids:
                results_dict['vector_raw_hits'] += 1
                results_dict['vector_mrr'] += 1/vector_doc_ids[doc_id]
                hit = True
                
            # if no hits, let's capture that
            if not hit:
                results_dict['total_misses'] += 1
                miss_info.append({'query': q, 'kw_response': kw_response, 'vector_response': vector_response})
        except Exception as e:
            print(f'Error due to {e}')
            continue
    
    #use raw counts to calculate final scores
    calc_hit_rate_scores(results_dict, search_type=['kw', 'vector'])
    calc_mrr_scores(results_dict, search_type=['kw', 'vector'])
    
    end = time.perf_counter() - start
    print(f'Total Processing Time: {round(end/60, 2)} minutes')
    record_results(results_dict, chunk_size, dir_outpath=dir_outpath)
    
    if include_miss_info:
        return results_dict, miss_info
    return results_dict

/usr/local/lib/python3.10/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:312: PydanticDeprecatedSince20: `json_encoders` is deprecated. See https://docs.pydantic.dev/2.9/concepts/serialization/#custom-serializers for alternatives. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(


### Run evaluation over golden dataset

In [6]:
#################
##  START CODE ##
#################
eval_results = retrieval_evaluation(golden_dataset, collection_name, retriever)

Queries: 100%|██████████| 100/100 [00:07<00:00, 12.95it/s]


Total Processing Time: 0.13 minutes

2024-12-06 15:30:20.318 | INFO     | src.preprocessor.preprocessing:save_as_json:109 - Data saved as json file here: ./eval_results/retrieval_eval_256_2024-12-06-15-30-20.json


In [7]:
print(eval_results)

{
    'n': 5,
    'Retriever': 'sentence-transformers/all-MiniLM-L6-v2',
    'chunk_size': 256,
    'query_props': ['content'],
    'kw_raw_hits': 65,
    'vector_raw_hits': 56,
    'kw_mrr': 0.54,
    'vector_mrr': 0.42,
    'total_misses': 21,
    'total_questions': 100,
    'kw_hit_rate': 0.65,
    'vector_hit_rate': 0.56
}

# Conclusion
***

We now have a way of measuring the performance of our retrieval system.  This will allow you to make tweaks/changes to the system and then be able to objectively tell whether or not the tweak/change improved or degraded its performance.  Here are a few things to consider going forward:  

- Keep in mind what the ulitmate goal of the system is that you are building.  For this course, we are trying to retrieve the most relevant documents as possible that will effectively address a user query, assuming the information is found within the corpus.  This means that we don't need pages and pages of relevant results, we actually only need the top 3-5, just enough to allow our Reader (the OpenAI LLM) to answer the user query.  This is an important point to be thinking about as you are making changes to the retrieval system.
- Feel free to set the `include_miss_info` param to `True`.  Doing so will return a list of both the keyword and vector responses that did not contain the relevant `doc_id` (a "total_miss" means the `doc_id` was not present in either the `kw_doc_ids` or the `vector_doc_ids`).  Take a look at the style of the queries being asked and compare them with the returned responses.  Why are those responses being returned?  Are they close to the intent of the query?
- Last but not least, you are now free to make changes to your system to improve the `hit_rate` and `mrr` scores.  If it were me, I'd start with switching out to a more performant [embedding model](https://huggingface.co/spaces/mteb/leaderboard).  There will be more opportunities to pick up some low hanging fruit, but we'll have to wait until the following week when hybrid search and Rerankers are introduced.  Whatever you do though, don't change params for the `SentenceSplitter` that you use for chunking the corpus.  Due to the way the golden dataset is derived, it's unfortunately dependent on those original `SentenceSplitter` settings remaining the same across evaluations. That is, of course, unless you want to build out your own golden dataset....

# *** OPTIONAL: Chunk Size Evaluation ***

In our initial Notebook we created a dataset with a chunk size of 256.  In order to evaluat the impact that chunk size has on retrieval for both search methods, it's a useful exercise to execute the `retrieval_evaluation` function on datasets of multiple chunk sizes.  In order to accomplish that follow these simple steps:
- Bust out the `create_dataset` function that you created in Assignment 1.4.  Create datasets of chunks lengths **128** and **512**.  **Ensure that you set the `chunk_overlap` param to zero for each run.  Golden datasets of corresponding chunks lengths have already been created for you in the `data/golden_datasets` directory.
- Index those datasets on Weaviate ensuring that you stick to the standard naming convention as discussed in Notebook 2 i.e `"Huberman_minilm_<chunk_size>"`
- Evaluate results using the chunk sizes as a parameter, see example code below:

### Sample code for automated chunk_size evaluation

In [8]:
all_results = []
for size in [128, 256, 512]:

    #load golden datasets
    data_path = f'../data/golden_datasets/golden_{size}.json'
    golden_dataset = FileIO().load_json(data_path)
    
    #assign collection name
    collection_name = f'Huberman_minilm_{size}'
    print(f'Running test on chunk size {size} on {collection_name} Collection')

    #get results by chunk_size
    results = retrieval_evaluation(golden_dataset, collection_name, retriever, query_properties=['content'], chunk_size=size)
    all_results.append(results)

Running test on chunk size 128 on Huberman_minilm_128 Collection

Queries:   0%|          | 0/100 [00:00<?, ?it/s]

Error due to Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
        status = StatusCode.UNKNOWN
        details = "could not find class Huberman_minilm_128 in schema"
        debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"could not find class 
Huberman_minilm_128 in schema", grpc_status:2, created_time:"2024-12-06T15:30:46.445733003+00:00"}"
>.

Queries:   1%|          | 1/100 [00:00<00:17,  5.58it/s]

Error due to Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
        status = StatusCode.UNKNOWN
        details = "could not find class Huberman_minilm_128 in schema"
        debug_error_string = "UNKNOWN:Error received from peer  
{created_time:"2024-12-06T15:30:46.627163316+00:00", grpc_status:2, grpc_message:"could not find class 
Huberman_minilm_128 in schema"}"
>.

...

Queries:  99%|█████████▉| 99/100 [00:17<00:00,  5.65it/s]

Error due to Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
        status = StatusCode.UNKNOWN
        details = "could not find class Huberman_minilm_512 in schema"
        debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"could not find class 
Huberman_minilm_512 in schema", grpc_status:2, created_time:"2024-12-06T15:31:28.358528162+00:00"}"
>.

Queries: 100%|██████████| 100/100 [00:17<00:00,  5.63it/s]


Total Processing Time: 0.3 minutes

2024-12-06 15:31:28.364 | INFO     | src.preprocessor.preprocessing:save_as_json:109 - Data saved as json file here: ./eval_results/retrieval_eval_512_2024-12-06-15-31-28.json


In [ ]:
print(all_results)